In [9]:
#Required for manipulating data

!pip install pandas
!pip install numpy
!pip install gspread
!pip install panel
!pip install hvplot
!pip install holoviews

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

#Required for building the interactive dashboard

import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [11]:
#Connect data source

import os

os.getcwd()

'C:\\Users\\Jake\\OneDrive\\Desktop'

In [12]:
os.chdir('C:\\Users\\Jake\\OneDrive\\Desktop')

df = pd.read_csv('Sample_Bank_Data.csv')

df

,Type,Product,Started Date,Completed Date,Description,Amount,Fee,Currency,State,Balance
0,DEBIT,Current,8/11/2023,8/11/2023,Vending Machine,-2.40,0,USD,Completed,1506.48
1,DEBIT,Current,8/14/2023,8/14/2023,ChristianaCare,-155.00,0,USD,Completed,1351.48
2,DEBIT,Current,8/14/2023,8/14/2023,ShakeShack,-33.56,0,USD,Completed,1317.92
3,ELEC,Current,8/14/2023,8/14/2023,CapitalOne,-317.14,0,USD,Completed,1000.78
4,DEBIT,Current,8/15/2023,8/15/2023,NordProducts,-18.28,0,USD,Completed,982.50
5,DEBIT,Current,8/15/2023,8/15/2023,TacoBell,-31.32,0,USD,Completed,951.18
6,ELEC,Current,8/16/2023,8/16/2023,USAA,-46.75,0,USD,Completed,904.43
7,DEBIT,Current,8/21/2023,8/21/2023,Amazon,-11.95,0,USD,Completed,892.48
8,DEBIT,Current,8/21/2023,8/21/2023,Amazon,-25.00,0,USD,Completed,867.48
9,DEBIT,Current,8/21/2023,8/21/2023,Parkmobile,-0.45,0,USD,Completed,867.03


In [13]:
#Clean data

df = df[['Completed Date', 'Description', 'Amount',]] #keep only desired columns
df['Description'] = df['Description'].map(str.lower) #lower case of descriptions

df = df.rename(columns={'Completed Date' : 'Date'}) #rename columns
df['Category'] = 'unassigned' #add category column

df.head()

,Date,Description,Amount,Category
0,8/11/2023,vending machine,-2.40,unassigned
1,8/14/2023,christianacare,-155.00,unassigned
2,8/14/2023,shakeshack,-33.56,unassigned
3,8/14/2023,capitalone,-317.14,unassigned
4,8/15/2023,nordproducts,-18.28,unassigned


In [ ]:
#define all categories
    # Bills
    # Food
    # Shopping
    # Transport
    # Entertainment
    # Services
    # Excluded

In [23]:
#Assign transactions to the correct category

# Bills

df['Category'] = np.where(df['Description'].str.contains(
    'christianacare|capitalone|usaa|zelle'), 'Bills', df['Category'])

# Food

df['Category'] = np.where(df['Description'].str.contains(
    'vending machine|tacobell|shakeshack|gridiron|starbucks|univlqr'), 'Food', df['Category'])

# Shopping

df['Category'] = np.where(df['Description'].str.contains(
    'amazon'), 'Shopping', df['Category'])

# Services

df['Category'] = np.where(df['Description'].str.contains(
    'coursera|empowerme|albert|apple|peacock|nordproducts|patreon'), 'Services', df['Category'])

# Entertainment

df['Category'] = np.where(df['Description'].str.contains('playstation|microsoft'), 'Entertainment', df['Category'])

# Transport

df['Category'] = np.where(df['Description'].str.contains('parkmobile'), 'Transport', df['Category'])

# Paycheck

df['Category'] = np.where(df['Description'].str.contains('powerhrg'), 'Paycheck', df['Category'])

# Convert the "Date" column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

pd.options.display.max_rows = 999
df.head(200)

,Date,Description,Amount,Category,Month,Year
0,2023-08-11,vending machine,-2.40,Food,8,2023
1,2023-08-14,christianacare,-155.00,Bills,8,2023
2,2023-08-14,shakeshack,-33.56,Food,8,2023
3,2023-08-14,capitalone,-317.14,Bills,8,2023
4,2023-08-15,nordproducts,-18.28,Services,8,2023
5,2023-08-15,tacobell,-31.32,Food,8,2023
6,2023-08-16,usaa,-46.75,Bills,8,2023
7,2023-08-21,amazon,-11.95,Shopping,8,2023
8,2023-08-21,amazon,-25.00,Shopping,8,2023
9,2023-08-21,parkmobile,-0.45,Transport,8,2023


In [24]:
# Check unassigned transactions and confirm all transactions are assigned to a category

unassigned = df.loc[df['Category'] == 'unassigned']
unassigned

,Date,Description,Amount,Category,Month,Year


In [ ]:
# Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses, and savings

In [32]:
# Get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

# Filter the dataframe to include only transactions from the lastest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

In [41]:
last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount']=last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount']=last_month_expenses['Amount'].str.replace('-', '')
last_month_expenses['Amount']=last_month_expenses['Amount'].astype('float')    #get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned|Paycheck") == False]
last_month_expenses = last_month_expenses.sort_values(by= 'Amount', ascending = False)   #sort values
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)         #round values

last_month_expenses

,Category,Amount
0,Bills,1024
2,Food,78
1,Entertainment,17
4,Services,5


In [42]:
last_month_expenses_tot = last_month_expenses['Amount'].sum()
last_month_expenses_tot

1124

In [43]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)
    
income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")
    
income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")
    
pn.Row(income_widget, recurring_expenses_widget,  monthly_expenses_widget, difference_widget).show()

Launching server at http://localhost:59979


In [44]:
# Create last month expenses bar chart

last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category',
    y='Amount',
    height=250,
    width=850,
    title="Last Month Expenses",
    ylim=(0, 500))

last_month_expenses_chart

:Bars   [Category]   (Amount)

In [61]:
# Create monthly expenses trend bar chart

df['Date'] = pd.to_datetime(df['Date'])               #convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')       #extract the month and year from the 'Date' column and create a new coulmn
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].str.replace('-', '')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Excluded|unassigned|Paycheck") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})   
                                                              
monthly_expenses_trend_by_cat                                                              

,Month-Year,Category,Amount
0,2023-08,Bills,1034
6,2023-09,Bills,1024
3,2023-08,Services,140
2,2023-08,Food,92
8,2023-09,Food,78
4,2023-08,Shopping,65
7,2023-09,Entertainment,17
1,2023-08,Entertainment,15
9,2023-09,Services,5
5,2023-08,Transport,0


In [60]:
# Define Panel Widget

select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Bills',
    'Services',
    'Food',
    'Entertainment',
    'Shopping',
    'Transport',
    #'Excluded'
])

select_category1

Select(options=['All', 'Bills', ...], value='All')

In [78]:
# Define plot function

def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot

# Define callback function

@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# Create layout

monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

C:\Users\Jake\AppData\Local\Temp\ipykernel_13796\1841426744.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()


Row
    [0] Select(options=['All', 'Bills', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, width=600)

In [67]:
# Create Summary Table

df = df[['Date', 'Category', 'Description', 'Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-', '')
df['Amount']=df['Amount'].astype('float')                #get absolute figures

df = df[df["Category"].str.contains("Excluded|Paycheck") ==False]     
df['Amount'] = df['Amount'].round().astype(int)
df

,Date,Category,Description,Amount
0,2023-08-11,Food,vending machine,2
1,2023-08-14,Bills,christianacare,155
2,2023-08-14,Food,shakeshack,34
3,2023-08-14,Bills,capitalone,317
4,2023-08-15,Services,nordproducts,18
5,2023-08-15,Food,tacobell,31
6,2023-08-16,Bills,usaa,47
7,2023-08-21,Shopping,amazon,12
8,2023-08-21,Shopping,amazon,25
9,2023-08-21,Transport,parkmobile,0


In [70]:
# Define a function to filter the dataframe based on the selected category

def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]

#Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget

select_category1.param.watch(update_summary_table, 'value')

summary_table

DataFrame(height=300, sizing_mode='fixed', value=         Date  ..., width=400)

In [83]:
# Create Final Dashboard

template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *If you can't manage your money, making more won't help*"),
        pn.pane.PNG('http://localhost:8888/view/vecteezy_pack-of-dollars-money-clipart-design-illustration_9391394_241.png', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,
            align='start',
            sizing_mode='stretch_width')])

template.show()

Launching server at http://localhost:53926
